# Exploring conformational space of "M1+2H2O" complexes

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Thu Apr 27 10:41:20 2017


In [2]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

def get_moldict_from_inplist(inplist):
    moldict = {}
    for i, inp in enumerate(inplist):
        mol = Chem.MolFromPDBFile(inp)
        name = os.path.basename(os.path.splitext(inp)[0]).rsplit("_", 0)[0]
        model = "_".join(os.path.basename(os.path.splitext(inp)[0]).rsplit("_", 2)[1:])
        moldict[name] = mol
    return moldict

In [3]:
# useful for later:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'
core_smiles = 'n1ccccc1'

m1 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal.sdf')
m1_crystal = templ_m1[0]

In [35]:
dir1 = '/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/results_starting_from_m1_2h2o_in/'
inps = []
for f in os.listdir(dir1):
    if f.endswith(".pdb"):
        inps.append(os.path.join(dir1, f))

mol = get_moldict_from_inplist(inps)

In [45]:
# align:
for key, m in mol.items():
    core_mol = m.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(m,mol["m1_2h2o_in_b_26"],atomMap=list(zip(core_mol,core_m1)))

In [46]:
p = py3Dmol.view(width=300,height=300)
for key, m in mol.items():
    mb = Chem.MolToMolBlock(m)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

## M1 + 2H2O

M1 + 2H2O structures were generated from the crystal geometry of isolated M1 macrocycle, to which we have added :

* 2 H2O molecules "inside" the macrocycle ("m1_2h2o_in", first from the left fig. below)

* 2 H2O molecules "outside" the macrocycle ("m1_2h2o_out", second from the left fig. below)

* 1 H2O molecule "inside" and 1 H2O molecule "outside" the macrocycle ("m1_2h2o_inout", third from the left fig. below)

manually in Avogadro software; then pre-optimized with MM implemented in Avogadro.

On figures below we present already pre-optimized "m1+2h2o" complexes, which serve as starting geometries for exploration of the conformational space.

In [33]:
%%html
<table>
  <tr>
    <td id="m1_2h2o_in" ></td>
    <td id="m1_2h2o_out" ></td>
    <td id="m1_2h2o_inout" ></td>    
  <tr>
    <td> m1_2h2o_in </td>
    <td> m1_2h2o_out  </td>  
    <td> m1_2h2o_inout  </td>     
  </tr>
</table>

,,
m1_2h2o_in,m1_2h2o_out,m1_2h2o_inout


In [34]:
m_in = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/m1_2h2o_in.xyz', 'r').read()
p_in = py3Dmol.view(width=300,height=300)
p_in.addModel(m_in,'xyz')
p_in.setStyle({'stick':{'radius':'0.15'}})
p_in.setBackgroundColor('0xeeeeee')
p_in.zoomTo()
p_in.insert('m1_2h2o_in')

In [35]:
m_out = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/m1_2h2o_out.xyz', 'r').read()
p_out = py3Dmol.view(width=300,height=300)
p_out.addModel(m_out,'xyz')
p_out.setStyle({'stick':{'radius':'0.15'}})
p_out.setBackgroundColor('0xeeeeee')
p_out.zoomTo()
p_out.insert('m1_2h2o_out')

In [36]:
m_inout = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/m1_2h2o_inout.xyz', 'r').read()
p_inout = py3Dmol.view(width=300,height=300)
p_inout.addModel(m_inout,'xyz')
p_inout.setStyle({'stick':{'radius':'0.15'}})
p_inout.setBackgroundColor('0xeeeeee')
p_inout.zoomTo()
p_inout.insert('m1_2h2o_inout')

### Conformers generated with the Balloon software:

In both cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

Below we will present aligned structures of macrocycle M1 conformers in "M1 + H2O" (without showing H2O molecules).

In [4]:
inps_m1_2h2o_in_b  = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/results_starting_from_m1_2h2o_in/*.sdf')
inps_m1_2h2o_out_b = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/results_starting_from_m1_2h2o_out/*.sdf')
inps_m1_2h2o_inout_b = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/results_starting_from_m1_2h2o_inout/*.sdf')

In [38]:
e_m1_2h2o_in_b  = grep_energies_from_sdf_outputs(inps_m1_2h2o_in_b)
e_m1_2h2o_out_b = grep_energies_from_sdf_outputs(inps_m1_2h2o_out_b)
e_m1_2h2o_inout_b = grep_energies_from_sdf_outputs(inps_m1_2h2o_inout_b)

In [46]:
%%html
<table>
  <tr>
    <td id="m1_2h2o_in_b" ></td>
    <td id="m1_2h2o_out_b"  ></td>
    <td id="m1_2h2o_inout_b"  ></td>    
  <tr>
    <td> m1_2h2o_in_b </td>
    <td> m1_2h2o_out_b  </td>  
    <td> m1_2h2o_inout_b  </td>     
  </tr>
</table>

,,
m1_2h2o_in_b,m1_2h2o_out_b,m1_2h2o_inout_b


In [40]:
# write conformers to dictionaries
allmol_m1_2h2o_in_b  = {}
allmol_m1_2h2o_out_b = {}
allmol_m1_2h2o_inout_b = {}
suppl_m1_2h2o_in_b  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/m1_2h2o_in_result.sdf')
suppl_m1_2h2o_out_b = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/m1_2h2o_out_result.sdf')
suppl_m1_2h2o_inout_b = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/balloon/m1_2h2o_inout_result.sdf')

for i, mol in enumerate(suppl_m1_2h2o_in_b):
    name = "m1_2h2o_in_b_" + str(i)
    allmol_m1_2h2o_in_b[name] = mol
for i, mol in enumerate(suppl_m1_2h2o_out_b):
    name = "m1_2h2o_out_b_" + str(i)
    allmol_m1_2h2o_out_b[name] = mol 
for i, mol in enumerate(suppl_m1_2h2o_inout_b):
    name = "m1_2h2o_inout_b_" + str(i)
    allmol_m1_2h2o_inout_b[name] = mol     

In [41]:
# align:
for key, mol in allmol_m1_2h2o_in_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
for key, mol in allmol_m1_2h2o_out_b.items():   
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))   
for key, mol in allmol_m1_2h2o_inout_b.items():   
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))      

In [47]:
# view:
p1_b_handles=[]

p1_b_in = py3Dmol.view(width=300,height=300)
for key, mol in allmol_m1_2h2o_in_b.items():
    mb = Chem.MolToMolBlock(mol)
    p1_b_in.addModel(mb,'sdf')
p1_b_in.setStyle({'stick':{'radius':'0.15'}})
p1_b_in.setBackgroundColor('0xeeeeee')
p1_b_in.zoomTo()    
p1_b_handles.append(p1_b_in)

p1_b_out = py3Dmol.view(width=300,height=300)
for key, mol in allmol_m1_2h2o_out_b.items():
    mb = Chem.MolToMolBlock(mol)
    p1_b_out.addModel(mb,'sdf')
p1_b_out.setStyle({'stick':{'radius':'0.15'}})
p1_b_out.setBackgroundColor('0xeeeeee')
p1_b_out.zoomTo()    
p1_b_handles.append(p1_b_out)

p1_b_inout = py3Dmol.view(width=300,height=300)
for key, mol in allmol_m1_2h2o_inout_b.items():
    mb = Chem.MolToMolBlock(mol)
    p1_b_inout.addModel(mb,'sdf')
p1_b_inout.setStyle({'stick':{'radius':'0.15'}})
p1_b_inout.setBackgroundColor('0xeeeeee')
p1_b_inout.zoomTo()    
p1_b_handles.append(p1_b_inout)

In [48]:
p1_b_handles[0].insert('m1_2h2o_in_b')

In [49]:
p1_b_handles[1].insert('m1_2h2o_out_b')

In [50]:
p1_b_handles[2].insert('m1_2h2o_inout_b')


### pre-screening

Some of the generated conformers are very much alike. To remove potential duplicates which were not "caught" by the Balloon program, we can compare the energies (preoptimized with MM) and the RMSD calculated against a reference structure (here: the crystal structure of M1). It does not matter against which structure we are aligning the conformers, since we are interested in relative RMS between them.

First let's print the energies and RMS values:


In [51]:
allmol_m1_2h2o_b = {}
allmol_m1_2h2o_b.update(allmol_m1_2h2o_in_b)
allmol_m1_2h2o_b.update(allmol_m1_2h2o_out_b)

energy_m1_2h2o_b = {}
energy_m1_2h2o_b.update(e_m1_2h2o_in_b)
energy_m1_2h2o_b.update(e_m1_2h2o_out_b)

rms_m1_2h2o_b = {}
for key, mol in allmol_m1_2h2o_b.items():
    rms_m1_2h2o_b[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))

In [52]:
rms_sorted = sorted(rms_m1_2h2o_b.items(), key=lambda x: x[1])
rms_thresh = 0.05

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m1_2h2o_b[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1_2h2o_b, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1_2h2o_b[mol]
    del energy_m1_2h2o_b[mol]
    del rms_m1_2h2o_b[mol]  

List sorted by RMS:
name = m1_2h2o_in_b_29, E = 54.890389, RMS = 0.226267
name = m1_2h2o_out_b_13, E = 54.689126, RMS = 0.234887
name = m1_2h2o_in_b_30, E = 55.355135, RMS = 0.240480
name = m1_2h2o_in_b_28, E = 54.757403, RMS = 0.259561
name = m1_2h2o_out_b_11, E = 53.750620, RMS = 0.306094
name = m1_2h2o_out_b_14, E = 56.054597, RMS = 0.346971
name = m1_2h2o_in_b_27, E = 54.056001, RMS = 0.354770
name = m1_2h2o_in_b_15, E = 51.625895, RMS = 0.407191
name = m1_2h2o_in_b_11, E = 51.001782, RMS = 0.436906
name = m1_2h2o_in_b_5, E = 50.805264, RMS = 0.454215
name = m1_2h2o_out_b_9, E = 51.992328, RMS = 0.467310
name = m1_2h2o_in_b_23, E = 52.184899, RMS = 0.471831
name = m1_2h2o_out_b_2, E = 50.811418, RMS = 0.482438
name = m1_2h2o_out_b_5, E = 50.923510, RMS = 0.486285
name = m1_2h2o_in_b_3, E = 50.699467, RMS = 0.488799
name = m1_2h2o_in_b_22, E = 52.036037, RMS = 0.490778
name = m1_2h2o_in_b_4, E = 50.783727, RMS = 0.504533
name = m1_2h2o_in_b_2, E = 50.612542, RMS = 0.507738
name = m1

In [53]:
for key, mol in allmol_m1_2h2o_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p_b = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1_2h2o_b.items():
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

### Conformers generated with the RDKit software:

In [4]:
inps_m1_2h2o_in_rdkit  = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/rdkit/results_starting_from_m1_2h2o_in/*.sdf')
inps_m1_2h2o_out_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/rdkit/results_starting_from_m1_2h2o_out/*.sdf')
inps_m1_2h2o_inout_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/rdkit/results_starting_from_m1_2h2o_inout/*.sdf')

In [5]:
e_m1_2h2o_in_rdkit = grep_energies_from_sdf_outputs(inps_m1_2h2o_in_rdkit)
e_m1_2h2o_out_rdkit = grep_energies_from_sdf_outputs(inps_m1_2h2o_out_rdkit)
e_m1_2h2o_inout_rdkit = grep_energies_from_sdf_outputs(inps_m1_2h2o_inout_rdkit)

In [6]:
# write conformers to dictionaries
allmol_m1_2h2o_in_rdkit = {}
suppl_m1_2h2o_in_rdkit  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/rdkit/m1_2h2o_in_result.sdf')
allmol_m1_2h2o_out_rdkit = {}
suppl_m1_2h2o_out_rdkit  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/rdkit/m1_2h2o_out_result.sdf')
allmol_m1_2h2o_inout_rdkit = {}
suppl_m1_2h2o_inout_rdkit  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/rdkit/m1_2h2o_inout_result.sdf')

for i, mol in enumerate(suppl_m1_2h2o_in_rdkit):
    name = "m1_2h2o_in_rdkit_" + str(i)
    allmol_m1_2h2o_in_rdkit[name] = mol  
for i, mol in enumerate(suppl_m1_2h2o_out_rdkit):
    name = "m1_2h2o_out_rdkit_" + str(i)
    allmol_m1_2h2o_out_rdkit[name] = mol  
for i, mol in enumerate(suppl_m1_2h2o_inout_rdkit):
    name = "m1_2h2o_inout_rdkit_" + str(i)
    allmol_m1_2h2o_inout_rdkit[name] = mol    

In [7]:
# align:
for key, mol in allmol_m1_2h2o_in_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
for key, mol in allmol_m1_2h2o_out_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles)) 
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
for key, mol in allmol_m1_2h2o_inout_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles)) 
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))    

In [8]:
%%html
<table>
  <tr>
    <td id="m1_2h2o_in_rdkit" ></td>
    <td id="m1_2h2o_out_rdkit"  ></td>
    <td id="m1_2h2o_inout_rdkit"  ></td>    
  <tr>
    <td> m1_2h2o_in_rdkit </td>
    <td> m1_2h2o_out_rdkit  </td>  
    <td> m1_2h2o_niout_rdkit  </td>    
  </tr>
</table>

,,
m1_2h2o_in_rdkit,m1_2h2o_out_rdkit,m1_2h2o_niout_rdkit


In [9]:
# view:
p1_2h2o_rdkit_handles=[]

p1_2h2o_in_rdkit = py3Dmol.view(width=300,height=300)
p1_2h2o_in_rdkit.removeAllModels()
for key, mol in allmol_m1_2h2o_in_rdkit.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_2h2o_in_rdkit.addModel(mb,'sdf')    
p1_2h2o_in_rdkit.setStyle({'stick':{'radius':'0.15'}})
p1_2h2o_in_rdkit.setBackgroundColor('0xeeeeee')
p1_2h2o_in_rdkit.zoomTo()
p1_2h2o_rdkit_handles.append(p1_2h2o_in_rdkit)

p1_2h2o_out_rdkit = py3Dmol.view(width=300,height=300)
p1_2h2o_out_rdkit.removeAllModels()
for key, mol in allmol_m1_2h2o_out_rdkit.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_2h2o_out_rdkit.addModel(mb,'sdf')    
p1_2h2o_out_rdkit.setStyle({'stick':{'radius':'0.15'}})
p1_2h2o_out_rdkit.setBackgroundColor('0xeeeeee')
p1_2h2o_out_rdkit.zoomTo()
p1_2h2o_rdkit_handles.append(p1_2h2o_out_rdkit)

p1_2h2o_inout_rdkit = py3Dmol.view(width=300,height=300)
p1_2h2o_inout_rdkit.removeAllModels()
for key, mol in allmol_m1_2h2o_inout_rdkit.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_2h2o_inout_rdkit.addModel(mb,'sdf')    
p1_2h2o_inout_rdkit.setStyle({'stick':{'radius':'0.15'}})
p1_2h2o_inout_rdkit.setBackgroundColor('0xeeeeee')
p1_2h2o_inout_rdkit.zoomTo()
p1_2h2o_rdkit_handles.append(p1_2h2o_inout_rdkit)

In [10]:
p1_2h2o_rdkit_handles[0].insert('m1_2h2o_in_rdkit')

In [11]:
p1_2h2o_rdkit_handles[1].insert('m1_2h2o_out_rdkit')

In [12]:
p1_2h2o_rdkit_handles[2].insert('m1_2h2o_inout_rdkit')

### pre-screening

In [13]:
allmol_m1_2h2o_rdkit = {}
allmol_m1_2h2o_rdkit.update(allmol_m1_2h2o_in_rdkit)
allmol_m1_2h2o_rdkit.update(allmol_m1_2h2o_out_rdkit)
allmol_m1_2h2o_rdkit.update(allmol_m1_2h2o_inout_rdkit)

energy_m1_2h2o_rdkit = {}
energy_m1_2h2o_rdkit.update(e_m1_2h2o_in_rdkit)
energy_m1_2h2o_rdkit.update(e_m1_2h2o_out_rdkit)
energy_m1_2h2o_rdkit.update(e_m1_2h2o_inout_rdkit)

rms_m1_2h2o_rdkit = {}
for key, mol in allmol_m1_2h2o_rdkit.items():
    rms_m1_2h2o_rdkit[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m1_rdkit[key], rms_m1_rdkit[key]))

In [14]:
rms_sorted = sorted(rms_m1_2h2o_rdkit.items(), key=lambda x: x[1])
rms_thresh = 0.05

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m1_2h2o_rdkit[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1_2h2o_rdkit, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1_2h2o_rdkit[mol]
    del energy_m1_2h2o_rdkit[mol]
    del rms_m1_2h2o_rdkit[mol] 

List sorted by RMS:
name = m1_2h2o_out_rdkit_38, E = 59.440000, RMS = 1.014580
name = m1_2h2o_in_rdkit_38, E = 59.440000, RMS = 1.014625
name = m1_2h2o_inout_rdkit_38, E = 59.440000, RMS = 1.014625
name = m1_2h2o_out_rdkit_6, E = 62.850000, RMS = 1.156328
name = m1_2h2o_inout_rdkit_6, E = 62.850000, RMS = 1.156332
name = m1_2h2o_in_rdkit_6, E = 62.850000, RMS = 1.156332
name = m1_2h2o_inout_rdkit_35, E = 62.850000, RMS = 1.156345
name = m1_2h2o_in_rdkit_35, E = 62.850000, RMS = 1.156345
name = m1_2h2o_out_rdkit_35, E = 62.850000, RMS = 1.156795
name = m1_2h2o_out_rdkit_12, E = 57.260000, RMS = 1.227433
name = m1_2h2o_in_rdkit_29, E = 60.250000, RMS = 1.256444
name = m1_2h2o_inout_rdkit_29, E = 60.250000, RMS = 1.256446
name = m1_2h2o_in_rdkit_54, E = 56.020000, RMS = 1.276147
name = m1_2h2o_out_rdkit_53, E = 56.020000, RMS = 1.276147
name = m1_2h2o_inout_rdkit_54, E = 56.020000, RMS = 1.276147
name = m1_2h2o_inout_rdkit_47, E = 57.260000, RMS = 1.280484
name = m1_2h2o_in_rdkit_47, E = 

In [15]:
for key, mol in allmol_m1_2h2o_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p_r = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1_2h2o_rdkit.items():
    mb = Chem.MolToMolBlock(mol)
    p_r.addModel(mb,'sdf')
p_r.setStyle({'stick':{'radius':'0.15'}})
p_r.setBackgroundColor('0xeeeeee')
p_r.zoomTo()
p_r.show()

In [16]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1-2h2o/list_selected_conformers_from_rdkit", "w") as f:
    for key, mol in allmol_m1_2h2o_rdkit.items():
        f.write(key+"\n")

### Summary

In [67]:
allmol_m1_2h2o = {}
allmol_m1_2h2o.update(allmol_m1_2h2o_b)
allmol_m1_2h2o.update(allmol_m1_2h2o_rdkit)

energy_m1_2h2o = {}
energy_m1_2h2o.update(energy_m1_2h2o_b)
energy_m1_2h2o.update(energy_m1_2h2o_rdkit)

rms_m1_2h2o = {}
for key, mol in allmol_m1_2h2o.items():
    rms_m1_2h2o[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m1[key], rms_m1[key]))

In [68]:
rms_sorted = sorted(rms_m1_2h2o.items(), key=lambda x: x[1])
rms_thresh = 0.1

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m1_2h2o, rms_thresh)

for mol in to_be_deleted:
    del allmol_m1_2h2o[mol]
    del energy_m1_2h2o[mol]
    del rms_m1_2h2o[mol]  

Conformers which will be deleted:
['m1_2h2o_out_b_13', 'm1_2h2o_in_b_3', 'm1_2h2o_in_b_2', 'm1_2h2o_in_b_6', 'm1_2h2o_in_b_7', 'm1_2h2o_out_b_3', 'm1_2h2o_in_b_17', 'm1_2h2o_inout_rdkit_38', 'm1_2h2o_out_b_10', 'm1_2h2o_out_rdkit_35', 'm1_2h2o_inout_rdkit_11', 'm1_2h2o_out_rdkit_11', 'm1_2h2o_out_rdkit_3', 'm1_2h2o_inout_rdkit_41', 'm1_2h2o_in_rdkit_12', 'm1_2h2o_out_rdkit_1', 'm1_2h2o_out_rdkit_13', 'm1_2h2o_out_rdkit_21', 'm1_2h2o_inout_rdkit_0']


In [69]:
print(len(allmol_m1_2h2o))

for key, mol in allmol_m1_2h2o.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1_2h2o.items():
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

10
